
# CityBikes


### Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import json
import pandas as pd

#### API request to CityBikes

In [2]:
"""
#API call for citybikes

base_url = r'http://api.citybik.es/v2/networks/'
response = requests.get(base_url)
"""

In [29]:
data = response.json()

In [30]:
#saving json file

with open('../citybike_data.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [8]:
#open json file

with open('../data/citybike_data.json', 'r') as json_file:
    data = json.load(json_file)

Initial data parsing done only to be lead to another link in order to request the actual data about the bikes stations that I wanted.

In [9]:
data.keys()

dict_keys(['networks'])

In [10]:
data['networks'][0]['location'].keys()

dict_keys(['city', 'country', 'latitude', 'longitude'])

In [11]:
data['networks'][0]['location']['country']

'RU'

In [12]:
#Iterating through 'data' to find information related to Toronto, ON

data_canada = []

for network in data['networks']:
    if network['location']['country'] == 'CA' and network['location']['city']== 'Toronto, ON':
        data_canada.append(network)

print(data_canada)

[{'company': ['Motivate International, Inc.', 'PBSC Urban Solutions'], 'gbfs_href': 'https://tor.publicbikesystem.net/ube/gbfs/v1/', 'href': '/v2/networks/bixi-toronto', 'id': 'bixi-toronto', 'location': {'city': 'Toronto, ON', 'country': 'CA', 'latitude': 43.653226, 'longitude': -79.3831843}, 'name': 'Bike Share Toronto'}]


<br>
Above data provided another link to retrieve data for Toronto bike stations.

In [13]:
"""
#Second API Request to get Toronto information

station_information = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/station_information').json()
station_status = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/station_status').json()
system_information = requests.get('https://toronto.publicbikesystem.net/customer/ube/gbfs/v1/en/system_information').json()
"""

In [21]:
#open json file

with open('../data/station_information.json', 'r') as json_file:
    station_information = json.load(json_file)

In [22]:
#open json file

with open('../data/station_status.json', 'r') as json_file:
    station_status = json.load(json_file)

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [23]:
#sifting through the data in order to understand how it is laid out.

station_information['data']['stations'][0]

{'station_id': '7000',
 'name': 'Fort York  Blvd / Capreol Ct',
 'physical_configuration': 'REGULAR',
 'lat': 43.639832,
 'lon': -79.395954,
 'altitude': 0.0,
 'address': 'Fort York  Blvd / Capreol Ct',
 'capacity': 35,
 'is_charging_station': False,
 'rental_methods': ['KEY', 'TRANSITCARD', 'CREDITCARD', 'PHONE'],
 'groups': [],
 'obcn': '647-643-9607',
 'nearby_distance': 500.0,
 '_ride_code_support': True,
 'rental_uris': {}}

In [37]:
#grabbing the required information from the station information data

bike_station_data = []

for station in station_information['data']['stations']:
    station_id= station['station_id']
    station_name = station['name']
    station_lat = station['lat']
    station_lon = station['lon']
    bike_station_data.append((station_id, station_name, station_lat, station_lon))

print(bike_station_data[:5])

[('7000', 'Fort York  Blvd / Capreol Ct', 43.639832, -79.395954), ('7001', 'Wellesley Station Green P', 43.66496415990742, -79.38355031526893), ('7002', 'St. George St / Bloor St W', 43.667333, -79.399429), ('7003', 'Madison Ave / Bloor St W', 43.667158, -79.402761), ('7004', 'University Ave / Elm St', 43.656518, -79.389099)]


<br>
Note: The bike numbers used for this project are a snapshop from when the data was acquired.

In [28]:
#collecting number of bikes from the station status data

bike_numbers = []

for bike in station_status['data']['stations']:
    station_id = bike['station_id']
    num_bikes = bike['num_bikes_available']
    bike_numbers.append((station_id, num_bikes))

print(bike_numbers[:5])

[('7000', 6), ('7001', 11), ('7002', 9), ('7003', 2), ('7004', 6)]


### Put your parsed results into a DataFrame.

In [29]:
station_df=pd.DataFrame(bike_station_data)
numbers_df=pd.DataFrame(bike_numbers)

In [30]:
#fixing column headers

station_df.columns = ['station_id', 'name', 'lat', 'lon' ]
numbers_df.columns = ['station_id', 'num_of_bikes']

In [31]:
numbers_df.head(1)

,station_id,num_of_bikes
0,7000,6


In [32]:
station_df.head(1)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954


In [33]:
#merging above dataframes into one

bike_df = pd.merge(station_df, numbers_df, on='station_id')

In [34]:
bike_df.head(1)

,station_id,name,lat,lon,num_of_bikes
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,6


In [36]:
#saving dataframe to csv

bike_df.to_csv(r'..\data\bike_df.csv', index=False)